In [111]:
import org.apache.spark.streaming.StreamingContext._
import org.apache.spark.streaming.{Seconds, StreamingContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SQLContext
import org.apache.spark.streaming.Duration
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.types
import spark.implicits._

import org.apache.spark.streaming.StreamingContext._
import org.apache.spark.streaming.{Seconds, StreamingContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SQLContext
import org.apache.spark.streaming.Duration
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.types
import spark.implicits._


In [112]:
!rm -r checkpoint


In [113]:
!mkdir checkpoint

In [114]:
val df = spark.read
    .json("input/*.json")

df: org.apache.spark.sql.DataFrame = [agent-phone: string, bath: string ... 17 more fields]


In [115]:
df.count()

res44: Long = 34


In [116]:
val schema = df.schema

schema: org.apache.spark.sql.types.StructType = StructType(StructField(agent-phone,StringType,true),StructField(bath,StringType,true),StructField(bedrooms,StringType,true),StructField(beds,StringType,true),StructField(data-est,StringType,true),StructField(data-lk,StringType,true),StructField(data-pk,StringType,true),StructField(data-pos,StringType,true),StructField(full_bathrooms,StringType,true),StructField(home_type,StringType,true),StructField(ldp-description-text,StringType,true),StructField(ldp-phone-link,StringType,true),StructField(price,StringType,true),StructField(property-info-address-citystatezip,StringType,true),StructField(property-info-address-main,StringType,true),StructField(sold date,StringType,true),StructField(sqft,StringType,true),StructField(status,StringType,true),...


In [130]:
val df = spark.readStream
    .format("json")
    .schema(schema)
    .option("multiLine", "false")
    .option("cleanSource", "archive")
    .option("sourceArchiveDir", f"input_archive")
    .load("input/*.json")

df: org.apache.spark.sql.DataFrame = [agent-phone: string, bath: string ... 17 more fields]


In [131]:
/*
df
    .writeStream
    .outputMode("append")
    .option("truncate", false)
    .format("console")
    .start()
    .awaitTermination()
    */

In [133]:
val df2 = df.writeStream
    .option("path", "output")
    .option("checkpointLocation", "checkpoint")
    .toTable("houses")

df2: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@b7797a9


In [134]:
Thread.sleep(4)
df2.stop()

In [135]:
val df3 = spark.read.table("houses")

df3: org.apache.spark.sql.DataFrame = [agent-phone: string, bath: string ... 17 more fields]


In [136]:
df3.show(10)

+--------------+-------------------+--------------------+-------+--------+-------------+-------------+--------+----------------+-------------+--------------------+--------------+--------+----------------------------------+--------------------------+------------+------------+-------------------+--------------------+
|   agent-phone|               bath|            bedrooms|   beds|data-est|      data-lk|      data-pk|data-pos|  full_bathrooms|    home_type|ldp-description-text|ldp-phone-link|   price|property-info-address-citystatezip|property-info-address-main|   sold date|        sqft|             status|                 url|
+--------------+-------------------+--------------------+-------+--------+-------------+-------------+--------+----------------+-------------+--------------------+--------------+--------+----------------------------------+--------------------------+------------+------------+-------------------+--------------------+
|          NULL|Henderson, NV 89011|Bedrooms and 